In [77]:
import pulp
import pandas as pd
import numpy as np 

In [78]:
# Load the distance matrix from an Excel file
excel_file = "distance_matrix.xlsx"  
dfs = pd.read_excel(excel_file, index_col=0, engine='openpyxl')

df = dfs.iloc[0:5,0:5].copy()

demand_i = np.random.rand(df.shape[1])*1000 #demand aslinya 
demands = {city : demand for city, demand in zip(df.columns,demand_i)}

In [100]:
# Define your parameters
facilities = df.columns.tolist()  # Facility locations
demand_points = df.index.tolist()  # Demand points
k = 3  # Number of facilities to open
maximum_coverage = 10000

In [86]:
# Create a PuLP linear programming problem
problem = pulp.LpProblem("MCLP", pulp.LpMaximize)

In [87]:
# Define binary variables for facility opening decision
facility_open = {j: pulp.LpVariable("x " + j, cat=pulp.LpBinary) for j in facilities}   #x
demand_point = {i: pulp.LpVariable("y " + i, cat=pulp.LpBinary) for i in demand_points}  #y
#demand_number = {i: pulp.LpVariable(i, cat=pulp.LpBinary) for i in demand_i} #a
# demand_fulfilled = pulp.LpVariable.dicts("demand_fulfilled", demand_points, cat='Binary')
# demand_fulfilled
#demand_point

In [88]:
# Define the objective function (maximize coverage)
problem += pulp.lpSum([demands[d] * demand_point[d] for d in demand_point])
problem


MCLP:
MAXIMIZE
995.6688970287411*y_KAPANEWON_GALUR + 264.99838942034404*y_KAPANEWON_LENDAH + 399.19977941125984*y_KAPANEWON_PANJATAN + 701.8332726696367*y_KAPANEWON_TEMON + 381.9918375968616*y_KAPANEWON_WATES + 0.0
VARIABLES
0 <= y_KAPANEWON_GALUR <= 1 Integer
0 <= y_KAPANEWON_LENDAH <= 1 Integer
0 <= y_KAPANEWON_PANJATAN <= 1 Integer
0 <= y_KAPANEWON_TEMON <= 1 Integer
0 <= y_KAPANEWON_WATES <= 1 Integer

In [89]:
# Set up constraints K (maximal SPKLU YANG DIBANGUN)
problem += pulp.lpSum([facility_open[f] for f in facilities]) == k

In [90]:
problem

MCLP:
MAXIMIZE
995.6688970287411*y_KAPANEWON_GALUR + 264.99838942034404*y_KAPANEWON_LENDAH + 399.19977941125984*y_KAPANEWON_PANJATAN + 701.8332726696367*y_KAPANEWON_TEMON + 381.9918375968616*y_KAPANEWON_WATES + 0.0
SUBJECT TO
_C1: x_KAPANEWON_GALUR + x_KAPANEWON_LENDAH + x_KAPANEWON_PANJATAN
 + x_KAPANEWON_TEMON + x_KAPANEWON_WATES = 3

VARIABLES
0 <= x_KAPANEWON_GALUR <= 1 Integer
0 <= x_KAPANEWON_LENDAH <= 1 Integer
0 <= x_KAPANEWON_PANJATAN <= 1 Integer
0 <= x_KAPANEWON_TEMON <= 1 Integer
0 <= x_KAPANEWON_WATES <= 1 Integer
0 <= y_KAPANEWON_GALUR <= 1 Integer
0 <= y_KAPANEWON_LENDAH <= 1 Integer
0 <= y_KAPANEWON_PANJATAN <= 1 Integer
0 <= y_KAPANEWON_TEMON <= 1 Integer
0 <= y_KAPANEWON_WATES <= 1 Integer

In [107]:
N = [[j for j in facilities if df[i][j] <= maximum_coverage] for i in demand_points]

# N = {i:[j for j in facilities if df[i][j] <= 10000] for i in demand_points}


{'KAPANEWON TEMON': ['KAPANEWON TEMON', 'KAPANEWON WATES'],
 'KAPANEWON WATES': ['KAPANEWON TEMON',
  'KAPANEWON WATES',
  'KAPANEWON PANJATAN'],
 'KAPANEWON PANJATAN': ['KAPANEWON WATES',
  'KAPANEWON PANJATAN',
  'KAPANEWON GALUR',
  'KAPANEWON LENDAH'],
 'KAPANEWON GALUR': ['KAPANEWON PANJATAN',
  'KAPANEWON GALUR',
  'KAPANEWON LENDAH'],
 'KAPANEWON LENDAH': ['KAPANEWON PANJATAN',
  'KAPANEWON GALUR',
  'KAPANEWON LENDAH']}

In [108]:
# Constraints for apalah 9tu 
for i in demand_points:
    problem += pulp.lpSum([facility_open[j] for j in N[i]]) >= demand_point[i]

In [109]:
problem

MCLP:
MAXIMIZE
995.6688970287411*y_KAPANEWON_GALUR + 264.99838942034404*y_KAPANEWON_LENDAH + 399.19977941125984*y_KAPANEWON_PANJATAN + 701.8332726696367*y_KAPANEWON_TEMON + 381.9918375968616*y_KAPANEWON_WATES + 0.0
SUBJECT TO
_C1: x_KAPANEWON_GALUR + x_KAPANEWON_LENDAH + x_KAPANEWON_PANJATAN
 + x_KAPANEWON_TEMON + x_KAPANEWON_WATES = 3

_C2: x_KAPANEWON_TEMON + x_KAPANEWON_WATES - y_KAPANEWON_TEMON >= 0

_C3: x_KAPANEWON_PANJATAN + x_KAPANEWON_TEMON + x_KAPANEWON_WATES
 - y_KAPANEWON_WATES >= 0

_C4: x_KAPANEWON_GALUR + x_KAPANEWON_LENDAH + x_KAPANEWON_PANJATAN
 + x_KAPANEWON_WATES - y_KAPANEWON_PANJATAN >= 0

_C5: x_KAPANEWON_GALUR + x_KAPANEWON_LENDAH + x_KAPANEWON_PANJATAN
 - y_KAPANEWON_GALUR >= 0

_C6: x_KAPANEWON_GALUR + x_KAPANEWON_LENDAH + x_KAPANEWON_PANJATAN
 - y_KAPANEWON_LENDAH >= 0

VARIABLES
0 <= x_KAPANEWON_GALUR <= 1 Integer
0 <= x_KAPANEWON_LENDAH <= 1 Integer
0 <= x_KAPANEWON_PANJATAN <= 1 Integer
0 <= x_KAPANEWON_TEMON <= 1 Integer
0 <= x_KAPANEWON_WATES <= 1 Integer

In [111]:
problem.solve()

1

In [112]:
facility_open

{'KAPANEWON TEMON': x_KAPANEWON_TEMON,
 'KAPANEWON WATES': x_KAPANEWON_WATES,
 'KAPANEWON PANJATAN': x_KAPANEWON_PANJATAN,
 'KAPANEWON GALUR': x_KAPANEWON_GALUR,
 'KAPANEWON LENDAH': x_KAPANEWON_LENDAH}

In [132]:
x_soln = np.array([facility_open[j].varValue for j in facilities])
x_soln

y_soln = np.array([demand_point[j].varValue for j in demand_points])
# And print some output
print (("Status:"), pulp.LpStatus[problem.status])
print ("Population Served is = ", pulp.value(problem.objective))
print("total demand : ", sum(list(demands.values())))
print ("x = ", x_soln)
print("y = ", y_soln)

Status: Optimal
Population Served is =  2743.6921761268436
total demand :  2743.6921761268436
x =  [1. 0. 0. 1. 1.]
y =  [1. 1. 1. 1. 1.]


2743.6921761268436